# Fig4-Bacteria spatial distributions

In [74]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import dask.array as da
from zarr.storage import DirectoryStore
from glob import glob
import pickle
from scipy.optimize import curve_fit


In [28]:
%matplotlib qt

In [84]:
"""plot style"""
linewidth = 4
mpl.rc('axes', linewidth=linewidth)
mpl.rc('font', family='Arial')
fontsize = 24


colors = {'no_inj': [0.8, 0.8, 0.8],
         'mock': [0.4, 0.4, 0.4],
         'e.coli': [0, 0.4, 0],
         'complete': [0, 0.8, 0],
         'bacteria': [0.8, 0, 0.8]}

def style_axes(ax, fontsize=24):
    plt.minorticks_off()
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.xaxis.set_tick_params(labelsize=20)
    ax.yaxis.set_tick_params(labelsize=20)
    for tick in ax.xaxis.get_major_ticks():
        tick.label1.set_fontsize(fontsize)
    for tick in ax.yaxis.get_major_ticks():
        tick.label1.set_fontsize(fontsize)
    plt.tight_layout()
    
    return ax


In [75]:
def hill(x, A, n, KD, offset):
    return A * (x ** n) / (KD ** n + x ** n) + offset

In [51]:
n_bins = 10
x = np.linspace(0, 1, n_bins)
start_id = np.where(x > 0.18)[0][0]


In [60]:
"""assemble and resample diptericin distributions"""
with open(r'/media/brandon/Data2/Brandon/fly_immune/Lightsheet_Z1/emL3_24_line_dist.pkl', 'rb') as f:
    line_dist_arr = pickle.load(f).values

# throw out the very first and last ap-bins, as these often have noise in them
line_dist_arr = line_dist_arr[:, 1:-2]
line_dist_arr /= 1e8

mean_line_dist = np.mean(line_dist_arr, axis=0)
std_line_dist = np.std(line_dist_arr, axis=0) / np.sqrt(len(line_dist_arr))

x = np.linspace(0, len(mean_line_dist), n_bins - start_id)
xp = np.arange(0, len(mean_line_dist))
mean_line_dist = np.interp(x, xp, mean_line_dist)
std_line_dist = np.interp(x, xp, std_line_dist)

In [37]:
"""assemble bacteria distributions"""
experiment_path = r'/media/brandon/Data2/Brandon/fly_immune/Lightsheet_Z1/2024_03_19_hml-gfp_ecoli-hs-dtom_bacteria_distribution'
density_name = 'bacteria.density.thresh_3.ome.zarr/1'
density_thresh = 0
available_volumes = []
summed_densities = []
larvae_dirs = glob(experiment_path + '/larva*')
for larvae_dir in larvae_dirs:
    if len(glob(larvae_dir + '/' + density_name)) > 0:
        im = da.from_zarr(DirectoryStore(larvae_dir + '/' + density_name))
        mask = im > density_thresh
        available_volumes.append(da.sum(da.sum(mask.squeeze(), axis=0), axis=1).compute())
        summed_densities.append(da.sum(da.sum(im.squeeze(), axis=0), axis=1).compute())
        
x = np.linspace(0, 1, n_bins)
sd_arr = np.zeros((len(summed_densities), n_bins))
voxel_volume =  2 * 2 * 0.325 * 0.325 * 10 ** -12
for i in range(len(sd_arr)):
    xp = np.linspace(0, 1, len(summed_densities[i]))
    sd_arr[i] = np.interp(x, xp, summed_densities[i] / available_volumes[i] / voxel_volume)
    
sd_arr /= 1e7
mean_bacteria = np.mean(sd_arr, axis=0)
std_bacteria = np.std(sd_arr, axis=0) / np.sqrt(len(sd_arr))

In [88]:
x = np.linspace(0, 1, n_bins)
fig, ax1 = plt.subplots(figsize=(10,5))
ax1.errorbar(x, mean_bacteria, std_bacteria, marker='o', 
             markerfacecolor=colors['bacteria'], markersize=24, 
             elinewidth=4, capsize=4, capthick=4, markeredgewidth=3, 
             markeredgecolor='k', ecolor='k', linewidth=4, color=colors['bacteria'])

ax1.set_xlabel('fraction of anterior-posterior axis', fontsize=fontsize)
ax1.set_ylabel('bacterial density \n($10^7$ cells / ml)', fontsize=fontsize, color=colors['bacteria'])

ax2 = ax1.twinx()
ax2.errorbar(x[start_id:], mean_line_dist, std_line_dist, marker='o', 
             markerfacecolor=colors['e.coli'], markersize=24, 
             elinewidth=4, capsize=4, capthick=4, markeredgewidth=3, 
             markeredgecolor='k', ecolor='k', linewidth=4, color=colors['e.coli'])
ax2.set_ylabel('diptericin-gfp fluorescence \nintensity ($10^8$ a.u.)', fontsize=fontsize, color=colors['e.coli'])

style_axes(ax1)
style_axes(ax2)
ax1.spines['top'].set_visible(True)
ax2.spines['right'].set_visible(True)

In [89]:
plt.savefig(r'/home/brandon/Documents/Code/diptericin-paper/figures/Fig4-Bacteria/bacteria_and_dpt_ap_dists.pdf')

## plot input-output function

In [97]:
plt.figure(figsize=(7,6))
x = mean_bacteria[start_id + 1:-1]
xerr = std_bacteria[start_id+1:-1]
y = mean_line_dist[1:-1]
yerr = std_line_dist[1:-1]

plt.errorbar(x, y, yerr=yerr, xerr=xerr, marker='o', markerfacecolor=colors['e.coli'], 
             markeredgewidth=4, markeredgecolor='k', ecolor='k', elinewidth=4, 
             capsize=4, capthick=4, linewidth=0, markersize=18, label='data')

p0 = (np.max(y) - np.min(y), 3, np.median(x), np.min(x))    # A, n, KD, offset
popt, pcov = curve_fit(hill, x, y, p0=p0, bounds=((0, 0, 0, 0), (np.max(y), np.inf, np.max(x), np.max(y))))
A, n, KD, offset = popt
xfit = np.linspace(1.5, 3, 1000)
yfit = hill(xfit, A, n, KD, offset)
plt.plot(xfit, yfit, '-', linewidth=4, color=colors['bacteria'], zorder=20, alpha=0.5, label='hill fit')

plt.xlabel('bacterial density ($10^7$ cells / ml)', fontsize=fontsize)
plt.ylabel('diptericin-gfp fluorescence \nintensity ($10^8$ a.u.)', fontsize=fontsize)
plt.legend(fontsize=0.75*fontsize, loc='lower right')
ax = style_axes(plt.gca())

In [98]:
plt.savefig(r'/home/brandon/Documents/Code/diptericin-paper/figures/Fig4-Bacteria/input-output-function.pdf')